In [7]:
import socks
import socket
socks.set_default_proxy(socks.SOCKS5, "127.0.0.1", 9150)
socket.socket = socks.socksocket
import requests
print (requests.get('http://icanhazip.com')).content

import json
import csv
import re
import sys 
import requests
reload(sys)
sys.setdefaultencoding('UTF-8')
import string
import time

t0=time.time()
#Definitions
global commsite, start, end,headers,headersReq
commsite = 'http://www.dailymail.co.uk/reader-comments/p/asset/readcomments/'
start = '?max=100&offset='
end ='&order=desc&rcCache=shout'
headers = 'id', 'dateCreated', 'message', 'assetId', 'assetUrl','assetCommentCount', 'userAlias','userLocation','assetHeadline',   'userIdentifier', 'voteRating', 'voteCount', 'hasProfilePicture'
headings = 'Comment ID', 'Comment Created Time', 'Text', 'Article ID', 'Article URL','assetCommentCount', 'User Alias','User Location','Article Headline',   'User ID', 'voteRating', 'voteCount', 'hasProfilePicture'
headersReq = {
    'Host': 'www.dailymail.co.uk',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
}


#Precompile RegExs
timeRegex = re.compile(r"(\d{4})-(\d\d)-(\d\d)T(\d\d):(\d\d):.+?Z")
pubRegex = re.compile(r"<meta property =\"article:published_time\" content=\"(\d{4})-(\d\d)-(\d\d)T(\d\d):(\d\d):.+?")

n=2

#UI
#URL = raw_input('Enter Full Daily Mail Article URL: ')
#article = re.search('\/article-(\d+)\/',URL).group(1)

#Setup CSV
outfile  = open('DMData4.csv', "wb")
writer = csv.writer(outfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
writer.writerow(headings+('length','artCreatedDate','Elapsed Time (minutes)','AverageWordCount','FacebookURL','Number of Parent Comments'))

#Isolate post ids from data.csv
fb_ids = []
#fb_idss= ['1234','1367474646645613', '1368068083252936', '1367411916651886', '1366269283432816']
fb_idss = ['1367926599933751', '1368806123179132', '1368980353161709', '1367871423272602', '1367930146600063', '1367849153274829', '1366875356705542', '1367879469938464', '1367337226659355', '1368722633187481', '1369132919813119', '1368533759873035', '1367140733345671', '1366125126780565', '1367785096614568', '1368981693161575', '1368695843190160', '1368436973216047', '1367599149966496', '1368289513230793', '1366312320095179', '1366683300058081', '1365968003462944', '1367806393279105', '1366323946760683', '1368981523161592', '1365771923482552', '1368201609906250', '1367222983337446', '1367991749927236', '1368334819892929', '1366901206702957', '1366948330031578', '1366721623387582', '1366562123403532', '1366051843454560', '1368803886512689', '1366878813371863', '1367064533353291', '1367760979950313', '1366201350106276', '1367979123261832', '1367614813298263', '1366700230056388', '1366630903396654', '1368392379887173', '1365954666797611', '1369119969814414', '1367258883333856', '1368760126517065', '1367735106619567', '1367209150005496', '1368622866530791', '1365995876793490', '1366613896731688', '1367610766632001', '1368488389877572', '1368314773228267', '1367267093333035', '1366432673416477', '1365862896806788', '1367337263326018', '1368404549885956', '1368350216558056', '1368457799880631', '1367554229970988', '1367011916691886', '1369019799824431', '1368916563168088', '1367503979976013', '1366377703421974', '1367958149930596', '1366956326697445', '1368166183243126', '1366892010037210', '1367825289943882', '1369003219826089', '1366897196703358', '1366884370037974', '1366307576762320', '1367790679947343', '1368618849864526', '1367920493267695', '1368798813179863', '1366722176720860', '1368083926584685', '1367179910008420', '1366042583455486', '1366249606768117', '1367092010017210', '1366716700054741', '1367780906614987', '1367541753305569', '1367229703336774', '1367147463344998', '1366903566702721', '1366352096757868', '1368654319860979', '1368864396506638', '1366087100117701', '1368156499910761', '1366168896776188', '1366977370028674', '1366298693429875', '1367744936618584', '1367723973287347', '1367422026650875', '1366060563453688', '1367515833308161', '1367249356668142', '1366446893415055', '1367357436657334', '1367819406611137', '1367345179991893', '1367980109928400', '1366744443385300', '1367712786621799', '1367484909977920', '1367277089998702', '1366385673421177', '1368518299874581', '1365896496803428', '1367773936615684']
with open('data.csv','rb') as csvfile:
    reader = csv.DictReader (csvfile)
    for row in reader:
        if row['is_post']=='1':
            match = re.search('._(.+)',row['id'])
            fb_ids.append(match.group(1))

def scrape(sec2,art2):
#Find Publish date
    global firstLine, currentLine, lastLine
    #print 'inside scrape loop'
    FacebookURL = [fb]
    URL = 'http://www.dailymail.co.uk/' + sec2 + '/article-' + art2
    req = requests.get(URL,headers=headersReq)
    lines = req.text
    match = re.search(pubRegex,lines)
    if match:
        pubtime=match.groups()
    pubTime = [match.group(3)+'/'+match.group(2)+'/'+match.group(1)+' '+match.group(4)+':'+match.group(5)]

    #First set of comments
    reqURL = commsite + art2
    req = requests.get (reqURL, headers = headersReq)
    html_source = req.text

    numOfComments = re.findall ('{"total":(\d+)',html_source)
    numOfParents = re.findall (',\"parentCommentsCount":(\d+),',html_source)
    nums = int(numOfComments[0])
    nump = int(numOfParents[0])
    numpList=[nump]
    
    lastLine = firstLine + nump-1
    #changed nums to nump in loop
    #Get rest of comments
    messagesLen=[]
    #n=exlCounter
    for i in range (0, nums,100): 
        comment = []
        req = requests.get (reqURL + start + str(i) + end, headers = headersReq)
        data = json.loads(req.text)
        for i in range(len(data['payload']['page'])):
            for header in headers:
                if header == 'dateCreated':
                    match = re.search(timeRegex, (data['payload']['page'][i]['dateCreated']))
                    if match:
                        (data['payload']['page'][i]['dateCreated']) = match.group(3)+'/'+match.group(2)+'/'+match.group(1)+' '+match.group(4)+':'+match.group(5)
                comment.append(data['payload']['page'][i].get(header,'N\\A'))
                if header == 'message':
                    words = str(data['payload']['page'][i]['message'])
                    words = words.translate(None, string.punctuation)
                    words = re.split(r'[^0-9A-Za-z]+',words.strip())
                    wordsCount=str(len(words)).split()
                    messagesLen.append(len(words))
                delta = []
            #print 'before delta'
            delta = ['=' + '(' + 'B' + str(currentLine) + '-' + 'O' + str(currentLine) + ')' + '*1440'] #MS Excel format
            #print 'before avg'
            averageWordCount = ['=average($N$'+str(firstLine) + ':' + 'N' + str(lastLine) + ')']
            #print 'after avg'
            comment=comment+wordsCount+pubTime+delta + averageWordCount+FacebookURL+numpList
            writer.writerow(comment)
            comment = []
            currentLine+=1
    firstLine = lastLine +1
    #exlCounter+= nump
            
firstLine = 2 #initiate excel counter for multifile extaction in one csv
currentLine = 2
lastLine = 2

counter = 1
successes = []
fails= []
for id in fb_idss:
    fb = 'https://www.facebook.com/DailyMail/posts/'+ id #str(1367031010023310)
    #print 'Downloading ' + fb
    req = requests.get(fb) #get DM post on fb  
    #print 'Searching daim.ai URL'
    match = re.search ('https:\/\/l\.facebook.com\/l\.php\?u\=http%3A%2F%2Fdailym\.ai\%2F(.+?)\&amp;',req.text) #get DM minisite article number from fb
    #print 'dailm.ai match found'
    if match:
        article1 = 'http://dailym.ai/' + match.group(1)
        req = requests.get(article1)
        #print 'Got dail.ai article'
        match = re.search ('<meta content=\"http:\/\/www\.dailymail\.co\.uk\/(.+)\/article-(.+?)\/',req.text) #get full article url
        #print 'Seraching for dailymail.co.uk URL'
        if match:
            sec2 = match.group(1)
            art2 = match.group(2)

            try:
                scrape (sec2,art2)
                print 'Done with id# ' + id + '.\t Status: SUCCESS. \t(',counter,'out of ' + str(len(fb_ids)) + ')'
                counter+=1
                successes.append(id)
            except:
                print 'Done with id# ' + id + '.\t Status: FAIL. Error within function \t(',counter,'out of ' + str(len(fb_ids)) + ')'
                counter+=1
                fails.append(id)

        else:
            print 'Done with id# ' + id + '.\t Status: FAIL. dailymail.co.uk URL Not found within dailym.ai \t(',counter,'out of ' + str(len(fb_ids)) + ')'
            counter+=1
            fails.append(id)

    else:   #        print (requests.get('http://icanhazip.com')).content
        print 'Done with id# ' + id + '.\t Status: FAIL. dailym.ai URL Not found within fb post \t(',counter,'out of ' + str(len(fb_ids)) + ')'
        counter+=1
        fails.append(id)
#        print (requests.get('http://icanhazip.com')).content    
outfile.close()

req.connection.close()

t1=time.time()

In [ ]:
req.text

In [ ]:
print len(fails)

In [3]:
print fb_ids

In [4]:
id

'1367891509937260'

In [ ]:
numOfComments = re.findall ('{"total":(\d+)',req.text)

In [5]:
numOfComments

NameError: name 'numOfComments' is not defined

In [8]:
req.text

u'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"><html><head><!--class fe.views.page_modules.Head-->\n<!--class fe.views.page_modules.Abe-->\n\n     \n     \n     <script type="text/javascript">var ABE={};ABE.startTime=(new Date).getTime(),ABE._rqueue=[],ABE.reveal=function(e){ABE._rqueue.push(e)}</script>\n     \n   \n\n<!--class fe.views.head_modules.Title-->\n<title>David Cameron QUITS in the wake of the historic Brexit vote in EU referendum | Daily Mail Online</title>\n<!--class fe.views.head_modules.MetaGoogleAds-->\n<!-- google_ad_section_start(name=s1) --><meta content="David,Cameron,QUITS,Prime,Minister,voters,Brexit,EU,referendum" name="keywords" /><meta content="David,Cameron,QUITS,Prime,Minister,voters,Brexit,EU,referendum" name="news_keywords" /><meta content="The Prime Minister said he accepted the verdict of the &apos;great democratic exercise&apos; which saw the\xa0Leave campaign triumph after stac

In [9]:
data

NameError: name 'data' is not defined

In [2]:
len(fails)

123

In [3]:
len(successes)

158

In [5]:
print fails

In [6]:
t1-t0

10787.417999982834

In [10]:
import time
t0 = time.time()
Scraped = {}

with open('DMData.csv','rb') as csvfile:
    reader = csv.DictReader (csvfile)
    for row in reader:
        Scraped[row['Article ID']]=Scraped.get(row['Article ID'],0)+1
                                                   
csvfile.close()
t1=time.time()

In [11]:
Scraped

{'3622204': 197,
 '3646510': 2,
 '3646875': 49,
 '3648991': 9,
 '3649964': 10,
 '3650132': 115,
 '3650396': 273,
 '3650512': 58,
 '3650795': 112,
 '3650820': 326,
 '3651277': 104,
 '3651458': 186,
 '3651857': 6,
 '3651860': 36,
 '3651968': 18,
 '3652001': 80,
 '3652133': 170,
 '3652143': 290,
 '3652261': 4,
 '3652340': 151,
 '3652438': 353,
 '3652484': 23,
 '3652624': 172,
 '3652637': 168,
 '3652648': 271,
 '3652749': 21,
 '3652768': 167,
 '3652878': 29,
 '3652901': 1627,
 '3652998': 63,
 '3653032': 41,
 '3653106': 166,
 '3653139': 232,
 '3653215': 56,
 '3653252': 68,
 '3653254': 113,
 '3653362': 234,
 '3653377': 119,
 '3653380': 1548,
 '3653404': 266,
 '3653406': 3,
 '3653438': 25,
 '3653442': 315,
 '3653492': 234,
 '3653495': 958,
 '3653497': 119,
 '3653526': 1351,
 '3653558': 161,
 '3653583': 124,
 '3653586': 109,
 '3653588': 642,
 '3653595': 361,
 '3653614': 122,
 '3653622': 626,
 '3653626': 1267,
 '3653645': 57,
 '3653758': 121,
 '3653763': 1,
 '3653766': 78,
 '3653782': 258,
 '36

In [12]:
counter = 0
for Scrape in Scraped:
    counter+=Scraped[Scrape]

In [13]:
counter

194773

In [14]:
len (Scraped)

221

In [15]:
fails

['1367926599933751',
 '1368806123179132',
 '1368980353161709',
 '1367871423272602',
 '1367930146600063',
 '1367849153274829',
 '1366875356705542',
 '1368722633187481',
 '1368981693161575',
 '1365771923482552',
 '1366901206702957',
 '1368803886512689',
 '1367760979950313',
 '1367614813298263',
 '1369019799824431',
 '1366892010037210',
 '1367825289943882',
 '1369003219826089',
 '1366897196703358',
 '1366884370037974',
 '1366307576762320',
 '1367790679947343',
 '1368618849864526',
 '1367920493267695',
 '1368798813179863',
 '1366722176720860',
 '1366716700054741',
 '1367780906614987',
 '1366903566702721',
 '1367422026650875',
 '1367980109928400',
 '1366385673421177']

In [16]:
10787 / 60

179